In [1]:
#Importamos las librerias necesarias para crear la red neuronal
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
import tensorflow as tf
#Leemos el csv
evaluation = pandas.read_csv('turkiye-student-evaluation_R_Specific.csv', header=0)

In [2]:
#Mostramos las 10 primeras lineas para ver que se ha leido bien
evaluation.head(10)

,instr,class,nb.repeat,attendance,difficulty,Q1,Q2,Q3,Q4,Q5,...,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28
1,1,2,1,0,4,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,1,2,1,2,4,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
4,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
5,1,2,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,1,2,1,3,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
7,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
8,1,2,1,1,3,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
9,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
10,1,2,1,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [3]:
#Seleccionamos los atributos 
atributos = evaluation.loc[:, 'class':'Q27']
atributos = atributos.to_numpy()
scaler = preprocessing.MinMaxScaler()
atributos = scaler.fit_transform(atributos)
print(atributos[5818,:])

[1.   0.   0.25 0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.  ]


In [4]:
#Seleccionamos el objetivo
objetivo = evaluation['Q28']
objetivo = evaluation['Q28'].to_numpy(dtype=np.float32)
print(objetivo)

[3. 3. 5. ... 5. 1. 1.]


In [5]:
#Separamos los datos que se usaran para entrenar y los que se usarán para evaluar los modelos
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos, objetivo, test_size=.15)

In [6]:
normalizador = keras.layers.Normalization()#Definimos un normalizador
normalizador.adapt(atributos_entrenamiento)
red_evaluation = keras.Sequential()#Creamos la red neuronal
red_evaluation.add(keras.layers.Input(shape=(31,)))#Definimos la capa de entrada con una neurona por cada atributo
red_evaluation.add(normalizador)#Aplicamos el normalizador
red_evaluation.add(keras.layers.Dense(70, activation='relu'))#Definimos una capa con 70 neuronas y función de activación relu
red_evaluation.add(keras.layers.Dense(70, activation='sigmoid'))#Definimos una capa de 70 neuronas y función de activación sigmoide
red_evaluation.add(keras.layers.Dense(1, activation='linear')) #Defnimios una capa de salida con función de activación lienal ya que los valores esperados son números lineales entre 0 y 5

In [7]:
red_evaluation.summary()#Mostramos un resumen de la red neuronal

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 31)               63        
 n)                                                              
                                                                 
 dense (Dense)               (None, 70)                2240      
                                                                 
 dense_1 (Dense)             (None, 70)                4970      
                                                                 
 dense_2 (Dense)             (None, 1)                 71        
                                                                 
Total params: 7,344
Trainable params: 7,281
Non-trainable params: 63
_________________________________________________________________


In [8]:

red_evaluation.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])#Compilamos la red neuronal definiendo el optimizaodr, la función de perdida y dos metricas
red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)#Entrenamos la red neuronal con 10 epocas y lotes de 256 elementos

Epoch 1/10
20/20 [==============================] - 1s 3ms/step - loss: 2.4015 - accuracy: 0.1290 - mae: 1.1389 
Epoch 2/10
20/20 [==============================] - 0s 2ms/step - loss: 0.6029 - accuracy: 0.1364 - mae: 0.5584
Epoch 3/10
20/20 [==============================] - 0s 2ms/step - loss: 0.4223 - accuracy: 0.1364 - mae: 0.4325
Epoch 4/10
20/20 [==============================] - 0s 2ms/step - loss: 0.3654 - accuracy: 0.1364 - mae: 0.3891
Epoch 5/10
20/20 [==============================] - 0s 2ms/step - loss: 0.3382 - accuracy: 0.1364 - mae: 0.3625
Epoch 6/10
20/20 [==============================] - 0s 2ms/step - loss: 0.3201 - accuracy: 0.1364 - mae: 0.3409
Epoch 7/10
20/20 [==============================] - 0s 2ms/step - loss: 0.3071 - accuracy: 0.1364 - mae: 0.3254
Epoch 8/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2970 - accuracy: 0.1364 - mae: 0.3143
Epoch 9/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2892 - accuracy: 0.1364 - mae

In [9]:
#Definimos cuanta perdida queremos tener y ejecutamos el siguiente bucle para entrenar la red hasta conseguir una perdida menor a la definida
#Para este conjunto de datos esto practicamente no es necesario pero así nos aseguramos de cometer un error muy pequeño
perdObj = 1.0
x=red_evaluation.evaluate(atributos_prueba, objetivo_prueba)
while x[0]>perdObj:
    red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)
    x=red_evaluation.evaluate(atributos_prueba, objetivo_prueba)
    
print(x[0])

28/28 [==============================] - 0s 1ms/step - loss: 0.2830 - accuracy: 0.1569 - mae: 0.2993
0.282995343208313


In [10]:
#Ahora que ya hemos conseguido una perdida menor a la que queriamos, evaluamos para volver a comprobar que la hemos conseguido
red_evaluation.evaluate(atributos_prueba, objetivo_prueba)

28/28 [==============================] - 0s 1ms/step - loss: 0.2830 - accuracy: 0.1569 - mae: 0.2993


[0.282995343208313, 0.1569301187992096, 0.29930031299591064]

In [11]:
#Ahora mismo comprobaremos manualmente si las predicciones de la red se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
X=atributos[5809]
predicciones = red_evaluation.predict(X)
print('Esperado 4:',predicciones)
X=atributos[5810]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5811]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5812]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5813]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5814]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5815]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5816]
predicciones = red_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos[5817]
predicciones = red_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos[5818]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5819]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)

1/1 [==============================] - 0s 97ms/step
Esperado 4: [[4.227154]]
1/1 [==============================] - 0s 26ms/step
Esperado 3: [[3.02421]]
1/1 [==============================] - 0s 27ms/step
Esperado 3: [[3.0289826]]
1/1 [==============================] - 0s 24ms/step
Esperado 1: [[1.1526291]]
1/1 [==============================] - 0s 25ms/step
Esperado 3: [[3.0142455]]
1/1 [==============================] - 0s 20ms/step
Esperado 1: [[1.3438317]]
1/1 [==============================] - 0s 26ms/step
Esperado 1: [[1.1526291]]
1/1 [==============================] - 0s 16ms/step
Esperado 5: [[4.8513656]]
1/1 [==============================] - 0s 24ms/step
Esperado 5: [[5.0478063]]
1/1 [==============================] - 0s 24ms/step
Esperado 1: [[1.158241]]
1/1 [==============================] - 0s 16ms/step
Esperado 1: [[1.158241]]


In [12]:
#Ahora trabajaremos con el otro modelo que será un random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
forest_evaluation = RandomForestRegressor() #Creamos el modelo
forest_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento)#Lo entrenamos
evaluaciones = forest_evaluation.predict(atributos_prueba)#Evaluamos
mse = mean_squared_error(objetivo_prueba, evaluaciones)#Calculamos la perdida
print("Error cuadrático medio (MSE) del modelo:", mse)
#Como este modelo suele tener una perdida muy pequeña tras entrenarlo una vez, no hacemos nada más para mejorarlo

Error cuadrático medio (MSE) del modelo: 0.21519976513455943


In [13]:
#Ahora mismo comprobaremos manualmente si las predicciones del random forest se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
atributos2 = atributos[:, np.newaxis]
X=atributos2[5809]
predicciones = forest_evaluation.predict(X)
print('Esperado 4:',predicciones)
X=atributos2[5810]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5811]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5812]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5813]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5814]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5815]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5816]
predicciones = forest_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos2[5817]
predicciones = forest_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos2[5818]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5819]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)

Esperado 4: [4.]
Esperado 3: [3.]
Esperado 3: [3.]
Esperado 1: [1.]
Esperado 3: [3.]
Esperado 1: [1.13]
Esperado 1: [1.]
Esperado 5: [5.]
Esperado 5: [5.]
Esperado 1: [1.]
Esperado 1: [1.]


In [14]:
#Importamos las librerias necesarias para crear otra red neuronal
#Volemos a importar por si se quiere ejecutar este fragmento sin ejecutar el anterior
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
import tensorflow as tf

#Leemos el csv
#bikes = pandas.read_csv('hour.csv', header=0)
bikes = pandas.read_csv('hour2.csv',header = 0)


In [15]:
#Mostramos las 10 primeras lineas para ver que se ha leido bién
bikes.head(10)

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
5,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
6,1,0,1,6,0,6,0,1,0.22,0.2727,0.80,0.0000,2,0,2
7,1,0,1,7,0,6,0,1,0.20,0.2576,0.86,0.0000,1,2,3
8,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0.0000,1,7,8
9,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0000,8,6,14


In [16]:
#Seleccionamos los atributos y los nomalizamos
attr = bikes.loc[:, 'season':'registered']
attr = attr.to_numpy()
scaler = preprocessing.MinMaxScaler()
attr = scaler.fit_transform(attr)

print(attr)

[[0.         0.         0.         ... 0.         0.00817439 0.01467269]
 [0.         0.         0.         ... 0.         0.02179837 0.03611738]
 [0.         0.         0.         ... 0.         0.01362398 0.03047404]
 ...
 [0.         1.         1.         ... 0.19301751 0.01907357 0.09367946]
 [0.         1.         1.         ... 0.15786999 0.03542234 0.05417607]
 [0.         1.         1.         ... 0.15786999 0.03269755 0.04176072]]


In [17]:
#Seleccionamos los objetivos
obj = bikes['cnt']
obj = bikes['cnt'].to_numpy(dtype=np.float32)
print(obj)

[16. 40. 32. ... 90. 61. 49.]


In [18]:
#Cogemos un 85% de los datos para entrenar y un 15% de los datos para evaluar el modelo
(attr_entrenamiento, attr_prueba,
 obj_entrenamiento, obj_prueba) = model_selection.train_test_split(
    attr, obj, test_size=.10)

In [19]:
normalizador = keras.layers.Normalization()#Definimos un normalizador
normalizador.adapt(attr_entrenamiento)
red_bikes = keras.Sequential()#Creamos la red neuronal
red_bikes.add(keras.layers.Input(shape=(14,))) #Añadimos la capa de entrada con 14 neuronas, una por cada atributo
red_bikes.add(normalizador)#Aplicamos un normalizador
red_bikes.add(keras.layers.Dense(100, activation='sigmoid'))#Añadimos una capa intermedia de 100 neuronas con función de activación sigmoide
red_bikes.add(keras.layers.Dense(100, activation='sigmoid'))#Añadimos una capa intermedia de 100 neuronas con función de activación sigmoide
red_bikes.add(keras.layers.Dense(1, activation='linear'))#Necesitamos una capa de salida linear ya que los valores objetivos son lineales

In [20]:
#Vemos un resumen de la red
red_bikes.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, 14)               29        
 ion)                                                            
                                                                 
 dense_3 (Dense)             (None, 100)               1500      
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 11,730
Trainable params: 11,701
Non-trainable params: 29
_________________________________________________________________


In [21]:
#Compilamos y entrenamos la red neuronal, buscamos minimizar la función de perdida
red_bikes.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])
red_bikes.fit(attr_entrenamiento, obj_entrenamiento,
                batch_size=256, epochs=10)

Epoch 1/10
62/62 [==============================] - 1s 2ms/step - loss: 11709.3330 - accuracy: 0.0091 - mae: 71.5937
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 2248.0715 - accuracy: 0.0092 - mae: 35.0991
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 879.8264 - accuracy: 0.0092 - mae: 22.9142
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 636.3483 - accuracy: 0.0092 - mae: 17.7560
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 602.1785 - accuracy: 0.0090 - mae: 16.4411
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 275.2636 - accuracy: 0.0092 - mae: 10.9759
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 348.8612 - accuracy: 0.0092 - mae: 13.6270
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 330.1798 - accuracy: 0.0092 - mae: 10.9941
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 151.1

In [22]:
#Definimos cuanta perdida queremos tener y ejecutamos el siguiente bucle para entrenar la red hasta conseguir una perdida menor a la definida
perdidaObj = 1.0
x=red_bikes.evaluate(attr_prueba, obj_prueba)
while x[0]>perdidaObj:
    red_bikes.fit(attr_entrenamiento, obj_entrenamiento,
                batch_size=256, epochs=10)
    x=red_bikes.evaluate(attr_prueba, obj_prueba)
    
#La forma más efectiva que hemos encontrado para mejorar nuestra red ha sido la de hacer este bulce ya que es la unica forma de asegurarnos que la red obtendrá la precisión que busquemos
print(x[0])

55/55 [==============================] - 0s 1ms/step - loss: 119.3348 - accuracy: 0.0081 - mae: 8.2107
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 95.6754 - accuracy: 0.0092 - mae: 7.6825
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 105.4293 - accuracy: 0.0092 - mae: 8.0932
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 100.0731 - accuracy: 0.0092 - mae: 7.4843
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 89.9830 - accuracy: 0.0092 - mae: 7.3871
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 112.1620 - accuracy: 0.0092 - mae: 8.3135
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 60.8816 - accuracy: 0.0092 - mae: 5.6943
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 65.8345 - accuracy: 0.0092 - mae: 6.4726
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 42.7361 - accuracy: 0.0092 - ma

62/62 [==============================] - 0s 2ms/step - loss: 12.7777 - accuracy: 0.0092 - mae: 2.9679
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 10.7787 - accuracy: 0.0092 - mae: 2.6578
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 17.8562 - accuracy: 0.0092 - mae: 3.4128
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 14.7986 - accuracy: 0.0081 - mae: 2.8954
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 20.1114 - accuracy: 0.0092 - mae: 3.6787
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 17.1733 - accuracy: 0.0092 - mae: 2.8884
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 2.6246 - accuracy: 0.0092 - mae: 1.1899
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 19.6400 - accuracy: 0.0092 - mae: 3.5302
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 5.6632 - accuracy: 0.0092 - mae: 1.

62/62 [==============================] - 0s 2ms/step - loss: 7.6339 - accuracy: 0.0092 - mae: 2.2049
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 5.0669 - accuracy: 0.0092 - mae: 1.7067
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 11.2999 - accuracy: 0.0092 - mae: 2.7402
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 1.7648 - accuracy: 0.0092 - mae: 1.0025
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 1.0385 - accuracy: 0.0092 - mae: 0.7635
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 5.7122 - accuracy: 0.0092 - mae: 1.7796
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 10.1061 - accuracy: 0.0092 - mae: 2.6392
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 15.0175 - accuracy: 0.0081 - mae: 3.3115
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 2.9804 - accuracy: 0.0092 - mae: 1.3263

55/55 [==============================] - 0s 1ms/step - loss: 1.5472 - accuracy: 0.0081 - mae: 0.9497
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 2.4218 - accuracy: 0.0092 - mae: 1.2336
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 6.8242 - accuracy: 0.0092 - mae: 2.0928
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 4.2940 - accuracy: 0.0092 - mae: 1.6907
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 2.7240 - accuracy: 0.0092 - mae: 1.3481
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 2.3400 - accuracy: 0.0092 - mae: 1.2893
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 4.2329 - accuracy: 0.0092 - mae: 1.7161
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 4.2404 - accuracy: 0.0092 - mae: 1.5927
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 1.1337 - accuracy: 0.0092 - mae: 0.8218
Epo

62/62 [==============================] - 0s 2ms/step - loss: 4.3510 - accuracy: 0.0092 - mae: 1.4083
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 1.6393 - accuracy: 0.0092 - mae: 0.9920
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 2.0535 - accuracy: 0.0092 - mae: 1.1153
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 0.6887 - accuracy: 0.0092 - mae: 0.6417
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 8.3877 - accuracy: 0.0081 - mae: 1.6972
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 7.6021 - accuracy: 0.0091 - mae: 2.1556
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 0.7732 - accuracy: 0.0092 - mae: 0.6802
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 0.8950 - accuracy: 0.0092 - mae: 0.7301
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 3.7250 - accuracy: 0.0092 - mae: 1.6879
Ep

In [23]:
#Ahora que ya hemos conseguido una perdida menor a la que queriamos, evaluamos para volver a comprobar que la hemos conseguido
red_bikes.evaluate(attr_prueba, obj_prueba)

55/55 [==============================] - 0s 1ms/step - loss: 0.8438 - accuracy: 0.0081 - mae: 0.6909


[0.8437759876251221, 0.00805523619055748, 0.6908671855926514]

In [24]:
#Ahora mismo comprobaremos manualmente si las predicciones de la red se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado

X=attr[17368]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 203:',predicciones)
X=attr[17369]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 247:',predicciones)
X=attr[17370]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 315:',predicciones)
X=attr[17371]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 214:',predicciones)
X=attr[17372]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 164:',predicciones)
X=attr[17373]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 122:',predicciones)
X=attr[17374]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 119:',predicciones)
X=attr[17375]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 89:',predicciones)
X=attr[17376]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 90:',predicciones)
X=attr[17377]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 61:',predicciones)
X=attr[17378]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 49:',predicciones)

Esperado 203: [[202.33147]]
Esperado 247: [[246.8391]]
Esperado 315: [[314.3512]]
Esperado 214: [[213.52046]]
Esperado 164: [[163.00238]]
Esperado 122: [[121.4043]]
Esperado 119: [[118.523026]]
Esperado 89: [[88.50641]]
Esperado 90: [[89.61465]]
Esperado 61: [[61.388153]]
Esperado 49: [[48.26579]]


In [25]:
#Ahora trabajaremos con el otro modelo que será un random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
forest_bikes = RandomForestRegressor()#Creamos el randomForest
forest_bikes.fit(attr_entrenamiento, obj_entrenamiento)#Lo entrenamos
evaluaciones = forest_bikes.predict(attr_prueba)#Evaluamos el modelo
mse = mean_squared_error(obj_prueba, evaluaciones)#Calculamos la perdida
print("Error cuadrático medio (MSE) del modelo:", mse)

Error cuadrático medio (MSE) del modelo: 6.765493958573074


In [26]:
#Ahora mismo comprobaremos manualmente si las predicciones del random forest se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
attr2 = attr[:, np.newaxis]
X=attr2[17368]
predicciones = forest_bikes.predict(X)
print('Esperado 203:',predicciones)
X=attr2[17369]
predicciones = forest_bikes.predict(X)
print('Esperado 247:',predicciones)
X=attr2[17370]
predicciones = forest_bikes.predict(X)
print('Esperado 315:',predicciones)
X=attr2[17371]
predicciones = forest_bikes.predict(X)
print('Esperado 214:',predicciones)
X=attr2[17372]
predicciones = forest_bikes.predict(X)
print('Esperado 164:',predicciones)
X=attr2[17373]
predicciones = forest_bikes.predict(X)
print('Esperado 122:',predicciones)
X=attr2[17374]
predicciones = forest_bikes.predict(X)
print('Esperado 119:',predicciones)
X=attr2[17375]
predicciones = forest_bikes.predict(X)
print('Esperado 89:',predicciones)
X=attr2[17376]
predicciones = forest_bikes.predict(X)
print('Esperado 90:',predicciones)
X=attr2[17377]
predicciones = forest_bikes.predict(X)
print('Esperado 61:',predicciones)
X=attr2[17378]
predicciones = forest_bikes.predict(X)
print('Esperado 49:',predicciones)

Esperado 203: [202.88]
Esperado 247: [247.08]
Esperado 315: [312.45]
Esperado 214: [214.14]
Esperado 164: [163.95]
Esperado 122: [122.78]
Esperado 119: [118.98]
Esperado 89: [89.05]
Esperado 90: [89.99]
Esperado 61: [61.18]
Esperado 49: [49.]


In [27]:
#Metodo LIME
import random
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPClassifier, MLPRegressor



"""
Partiremos de este seudocódigo para implementar el metodo
N es el n´umero de permutaciones a realizar
f es el modelo a explicar
X′ ← {} muestras perturbadas
R ← {} representaciones
W ← {} las distancias entre la muestra x y sus perturbaciones
for 1
to
N do
Selecciona k atributos aleatoriamente
x′ ← una perturbaci´on de x donde se perturban los k atributos anteriores.
w ← la distancia entre x y x′
r ← la representaci´on de x′ respecto a x
X′ ← X′ ∪ x′
R ← R ∪ r
W ← R ∪ w
end for
Y ′ ← f(X′) las predicciones de las perturbaciones
G ← modelo ridge entrenado con R para predecir Y ′ y ponderando cada
muestra con W
return los par´ametros de G

"""

def explain_model(f, x, N,M):
    #f es el modelo a explicar
    #X es una lista de ejemplos a los que se aplicará LIME
    #N es el número de permutaciones a realizar
    #M contiene todos los ejemplos
    Xi = [] #Aqui guardaremos las muestras perturbadas
    R = []  #Aqui guardaremos las representaciones
    W = []  #Aqui guradaremos las distancias
    for i in range(N):
        k = random.randint(1, len(x)) #Escojo un número aletario que representa el número de los atributos a seleccionar
        perturbed_x = x.copy() #copio la muestra original
        for j in range(k): 
            perturbed_attr = random.randint(0,len(x)-1)#random.choice(list(x.keys())) Voy escojiento los atributos que perturbare de forma aleatoria
            mx = abs(max([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor máximo
            mn = abs(min([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor mínimo
            perturbed_x[perturbed_attr] = random.uniform(mn, mx) # pertubo el atributo con un valor aleatorio ente mn y mx para que el valor esté acotado
        w = abs(sum([x[attr] - perturbed_x[attr] for attr in range(0,len(x)-1)])) #Calculo la distancia entre el original y el perturbado
        r = [0  if perturbed_x[attr] == x[attr] else 1 for attr in range(0,len(x)-1)] #Calculo la representación del perturbado respecto al original utilizando un operador ternario
        Xi.append(perturbed_x) #Acumulo en la lista los perturbados
        R.append(r)#Acumulamos en la lista de representaciones
        W.append(w)#Añadimos las distancias a la lista
    Y_perturbed = []#Aqui guardaremos las predicciones de las perturbaciones
    for i in range(len(Xi)):
        y = 1
            #En los siguiente if y else comprobamos el tipo de modelo que tenemos ya que dependiendo del tipo necesita recibir los atributos de una forma u otra
        if isinstance(f, keras.Sequential):#Comprueba si el modelo es una red neuronal
            xi = Xi[i]
            y = f.predict(xi,verbose=0)#Hacemos la prediccion con verbose=0 para no cargar la salida de lineas generadas por keras
        else: #Si no, es un randomForest
            xi = Xi[i]
            array = np.array(xi)
            xi = array.reshape(1, -1)
            y = f.predict(xi)#RandomForest no tiene verbose porque no genera nada
                
            
        Y_perturbed.append(y) #Aplico el modelo f a los ejemplos perturbados
            
    Y_perturbed = np.squeeze(Y_perturbed) #Por alguna razón aparece el error Found array with dim 3. Estimator expected <= 2" y lo arreglamos con esta linea que quita una dimensión al array
    G = Ridge()
    G.fit(R, Y_perturbed, sample_weight=W)
    return G.get_params() #Devolvemos los parametros


In [28]:
#Hacemos varias pruebas para comprobar que funciona el metodo LIME
atributivos = atributos[5809]

explain_model(red_evaluation,atributivos,10,atributos)

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': 'deprecated',
 'positive': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.001}

In [29]:
atributivos = atributos[5809]

explain_model(forest_evaluation,atributivos,10,atributos)

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': 'deprecated',
 'positive': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.001}

In [30]:
attri = attr[17378]

explain_model(red_bikes,attri,10,attr)

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': 'deprecated',
 'positive': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.001}

In [31]:
attri = attr[17378]

explain_model(forest_bikes,attri,10,attr)

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': 'deprecated',
 'positive': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.001}

In [32]:
import numpy as np


def identidad(x1,x2,exp1,exp2):
    #Definición: El principio de identidad establece que objetos idénticos deben recibir explicaciones idénticas.

    res = False #Si los objetos no son identicos no hay que valuar
    distance = np.linalg.norm(x1 - x2)#Calcula las distancias entre los dos ejemplos
    if(distance ==0):   #Solo comprobamos si las explicaciones sean identicas si los objetos son identicos
        res = exp1== exp2#Comprobamos si ambas explicaciones son iguales
    return res

In [38]:

import numpy as np


def separabilidad(x1,x2,exp1,exp2):
#Definición:#Separabilidad: Objetos no idénticos no pueden tener explicaciones idénticas. Para simplificar, cada característica 
#tiene un nivel mínimo de importancia, positivo o negativo, en las predicciones.

    res = True
    distance = np.linalg.norm(x1 - x2)#Calcula las distancias entre los dos ejemplos
    if(distance >0):   #Si la distancia es distinta de cero
        res = not(exp1== exp2)#Si las explicaciones son distintas se cumple la separabilidad
    
    return res

In [ ]:
import numpy as np

def estabilidad(X, epsilon):
    """
    Calcula la métrica de Estabilidad para el método LIME.

    Args:
        X (ndarray): Matriz de instancias de datos de forma (n, m), donde n es el número de instancias y m es el número de características.
        epsilon (float): Valor de perturbación.

    Returns:
        bool: True si se cumple la métrica de Estabilidad para todas las características, False en caso contrario.
    """

    n = X.shape[0]  # número de instancias
    m = X.shape[1]  # número de características

    # Calcula las distancias entre todas las parejas de instancias
    distances = np.linalg.norm(X[:, np.newaxis] - X, axis=2)

    # Calcula las distancias entre las instancias perturbadas
    perturbed_distances = np.linalg.norm(epsilon * X[:, np.newaxis] - epsilon * X, axis=2)

    # Verifica si se cumple la métrica de Estabilidad para todas las características
    for i in range(m):
        max_distance = np.max(distances[:, i])
        min_perturbed_distance = np.min(perturbed_distances[:, i])

        if min_perturbed_distance <= max_distance:
            return False

    return True


In [ ]:
#METODO ESTABILIDAD
# Creamos las instancias de datos y sus respectivas explicaciones LIME
x = np.array([[0.5, 0.7, 0.1, 0.3], [0.3, 0.9, 0.2, 0.1], [0.1, 0.2, 0.8, 0.6]])
ex = np.array([[0.2, 0.3, 0.1, 0.4], [0.3, 0.4, 0.5, 0.1], [0.1, 0.3, 0.6, 0.2]])

# Creamos las distancias entre las instancias de datos y sus respectivas explicaciones LIME
data_distances = cdist(x, x[0].reshape(1,-1))
exp_distances = cdist(ex, ex[0].reshape(1,-1))

# Calculamos el coeficiente de correlación de Pearson entre las distancias
correlation, _ = pearsonr(data_distances.flatten(), exp_distances.flatten())

# Verificamos si la métrica Estabilidad se cumple
if correlation > 0:
    print("La métrica Estabilidad se cumple para este conjunto de instancias de datos y sus explicaciones LIME correspondientes.")
else:
    print("La métrica Estabilidad no se cumple para este conjunto de instancias de datos y sus explicaciones LIME correspondientes.")


In [ ]:
import numpy as np

def coherencia(X, epsilon):
    """
    Calcula la métrica de Coherencia para el método LIME.

    Args:
        X (ndarray): Matriz de instancias de datos de forma (n, m), donde n es el número de instancias y m es el número de características.
        epsilon (float): Valor de perturbación.

    Returns:
        float: Valor de la métrica de Coherencia.
    """

    n = X.shape[0]  # número de instancias
    m = X.shape[1]  # número de características

    alpha_values = np.zeros(m)  # arreglo para almacenar los valores de alpha

    # Calcula las distancias entre todas las parejas de instancias
    distances = np.linalg.norm(X[:, np.newaxis] - X, axis=2)

    # Calcula las distancias entre las instancias perturbadas
    perturbed_distances = np.linalg.norm(epsilon * X[:, np.newaxis] - epsilon * X, axis=2)

    # Calcula los valores de alpha para cada característica
    for i in range(m):
        differences = np.exp(-distances[:, i]) - np.exp(-perturbed_distances[:, i])
        alpha_values[i] = np.mean(differences) / np.std(differences)

    return alpha_values


In [ ]:
#METODO COHERENCIA
# Creamos las instancias de datos y sus respectivas explicaciones LIME
x = np.array([[0.5, 0.7, 0.1, 0.3], [0.3, 0.9, 0.2, 0.1], [0.1, 0.2, 0.8, 0.6]])
ex = np.array([[0.2, 0.3, 0.1, 0.4], [0.3, 0.4, 0.5, 0.1], [0.1, 0.3, 0.6, 0.2]])

# Calculamos los pesos de las instancias de datos y sus respectivas explicaciones LIME
data_weights = np.exp(-np.linalg.norm(x - x.mean(axis=0), axis=1))
exp_weights = np.exp(-np.linalg.norm(ex - ex.mean(axis=0), axis=1))

# Calculamos los logaritmos naturales de los pesos y los sumamos
log_sum_data = np.sum(np.log(data_weights))
log_sum_exp = np.sum(np.log(exp_weights))

# Calculamos la métrica Coherencia
alpha = np.exp(log_sum_data - log_sum_exp + lambertw(0.0, -np.exp(log_sum_data - log_sum_exp)).real)
alpha /= len(x)

# Imprimimos el valor de la métrica Coherencia
print("El valor de la métrica Coherencia es:", alpha)


In [ ]:
import numpy as np

def completitud(alpha_values):
    """
    Calcula la métrica de Completitud para el método LIME.

    Args:
        alpha_values (ndarray): Arreglo de valores de alpha calculados para cada característica.

    Returns:
        float: Valor de la métrica de Completitud.
    """

    alpha_exp = np.exp(alpha_values)  # Calcula la exponencial de los valores de alpha
    completeness = np.sum(alpha_exp) / (np.sum(alpha_exp) + 1)

    return completeness


In [ ]:
#METODO COMPLETITUD
# Creamos las instancias de datos y sus respectivas explicaciones LIME
x = np.array([[0.5, 0.7, 0.1, 0.3], [0.3, 0.9, 0.2, 0.1], [0.1, 0.2, 0.8, 0.6]])
ex = np.array([[0.2, 0.3, 0.1, 0.4], [0.3, 0.4, 0.5, 0.1], [0.1, 0.3, 0.6, 0.2]])

# Calculamos los pesos de las instancias de datos y sus respectivas explicaciones LIME
data_weights = np.exp(-np.linalg.norm(x - x.mean(axis=0), axis=1))
exp_weights = np.exp(-np.linalg.norm(ex - ex.mean(axis=0), axis=1))

# Calculamos los valores de ei y pi
ei = np.sum(exp_weights)
pi = np.sum(data_weights)

# Calculamos la métrica Completitud
gamma = ei / pi

# Imprimimos el valor de la métrica Completitud
print("El valor de la métrica Completitud es:", gamma)


In [ ]:
import numpy as np

def congruencia(alpha_values):
    """
    Calcula la métrica de Congruencia para el método LIME.

    Args:
        alpha_values (ndarray): Arreglo de valores de alpha calculados para cada característica.

    Returns:
        float: Valor de la métrica de Congruencia.
    """

    alpha_avg = np.mean(alpha_values)  # Coherencia promedio
    n = alpha_values.shape[0]  # Número de características

    # Calcula la métrica de Congruencia
    delta = np.sum(np.square(alpha_values - alpha_avg)) / n

    return delta


In [ ]:
#METODO CONGRUENCIA
# Creamos una lista de valores de coherencia para un conjunto de N muestras
alpha_values = [0.8, 0.6, 0.7, 0.9, 0.5]

# Calculamos el valor promedio de la coherencia
alpha = np.mean(alpha_values)

# Creamos una lista de valores de alpha_i para cada instancia de datos
alpha_i_values = [0.9, 0.7, 0.6, 0.8, 0.4]

# Calculamos el número de instancias de datos
N = len(alpha_i_values)

# Calculamos la suma de las diferencias cuadráticas entre alpha_i y alpha
diff_squares_sum = np.sum((np.array(alpha_i_values) - alpha) ** 2)

# Calculamos la métrica Congruencia
delta = np.sqrt((diff_squares_sum / N) / alpha)

# Imprimimos el valor de la métrica Congruencia
print("El valor de la métrica Congruencia es:", delta)


In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score

def selectividad(X, y, model):
    """
    Calcula la métrica de Selectividad para el método LIME.

    Args:
        X (ndarray): Matriz de instancias de datos de forma (n, m), donde n es el número de instancias y m es el número de características.
        y (ndarray): Vector de etiquetas de forma (n,), donde n es el número de instancias.
        model: Modelo de aprendizaje automático ajustado sobre los datos completos.

    Returns:
        float: Área bajo la curva (AUC) que mide la selectividad al eliminar características una por una.
    """

    n = X.shape[0]  # número de instancias
    m = X.shape[1]  # número de características

    auc_scores = []  # lista para almacenar los resultados del AUC

    # Realiza las predicciones sobre los datos completos
    y_pred_base = model.predict_proba(X)[:, 1]

    # Calcula el AUC para el modelo sobre los datos completos
    auc_base = roc_auc_score(y, y_pred_base)

    # Itera sobre cada característica y calcula el AUC al eliminarla
    for i in range(m):
        X_copy = X.copy()
        X_copy[:, i] = 0  # Establece en cero la i-ésima característica

        # Realiza las predicciones con la característica eliminada
        y_pred = model.predict_proba(X_copy)[:, 1]

        # Calcula el AUC y lo agrega a la lista de resultados
        auc_score = roc_auc_score(y, y_pred)
        auc_scores.append(auc_score)

    # Calcula el AUC promedio para la selectividad
    auc_selectivity = np.mean(auc_scores)

    return auc_selectivity


In [ ]:
#METODO SELECTIVIDAD
# Cargamos el dataset de cáncer de mama de scikit-learn
data = load_breast_cancer()

# Definimos las características y la variable objetivo
X = data['data']
y = data['target']

# Creamos un modelo de regresión logística
model = LogisticRegression()

# Entrenamos el modelo
model.fit(X, y)

# Obtenemos las predicciones del modelo en el conjunto de datos completo
y_pred = model.predict_proba(X)[:, 1]

# Calculamos el AUC para el modelo entrenado en el conjunto de datos completo
auc_full = roc_auc_score(y, y_pred)

# Ordenamos las características de más a menos relevantes según el modelo
idx_sorted = np.argsort(np.abs(model.coef_))[0][::-1]

# Creamos una lista para almacenar los AUC para cada característica eliminada
auc_scores = []

# Iteramos sobre las características de más a menos relevantes según el modelo
for i in range(X.shape[1]):
    # Establecemos la característica i en cero
    X_test = X.copy()
    X_test[:, idx_sorted[i]] = 0
    
    # Obtenemos las predicciones del modelo en el conjunto de datos con la característica i eliminada
    y_pred_test = model.predict_proba(X_test)[:, 1]
    
    # Calculamos el AUC para el modelo entrenado en el conjunto de datos con la característica i eliminada
    auc_test = roc_auc_score(y, y_pred_test)
    
    # Almacenamos el AUC para la característica i eliminada en la lista de AUC
    auc_scores.append(auc_test)

# Calculamos la métrica Selectividad como el área bajo la curva ROC para el gráfico de AUC vs características eliminadas
selectivity = np.trapz(auc_scores, dx=1)

# Imprimimos el valor de la métrica Selectividad
print("El valor de la métrica Selectividad es:", selectivity)


In [36]:
#Comprobaremos la metrica identidad
#Para comprobarla recorreremos 256 ejemplos en los cuatro modelos y para cada ejemplo en cada modelo generaremos la explicacion dos veces y comprobaremos que es la misma explicación
atributivos = atributos[5553:5809,:]
sonIguales = True
for x in atributivos:
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    exp2 = explain_model(forest_evaluation,x,2,atributos)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break
if(sonIguales):
    print("El modelo RandomForest de evaluación cumple la metrica identidad")
else:
    print("El modelo RandomForest de evaluación no cumple la metrica identidad")

sonIguales=True
for x in atributivos:
    exp1 = explain_model(red_evaluation,x,2,atributos)
    exp2 = explain_model(red_evaluation,x,2,atributos)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break

if(sonIguales):
    print("El modelo Red neuronal de evaluación cumple la metrica identidad")
else:
    print("El modelo Red neuronal de evaluación no cumple la metrica identidad")

X=attr[17122:17378,:]
sonIguales=True
for x in X:
    exp1 = explain_model(red_bikes,x,2,attr)
    exp2 = explain_model(red_bikes,x,2,attr)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break
    
if(sonIguales):
    print("El modelo Red neuronal de bikes cumple la metrica identidad")
else:
    print("El modelo Red neuronal de bikes no cumple la metrica identidad")
    
sonIguales=True
for x in X:
    exp1 = explain_model(forest_bikes,x,2,attr)
    exp2 = explain_model(forest_bikes,x,2,attr)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break
    
if(sonIguales):
    print("El modelo RandomForest de bikes cumple la metrica identidad")
else:
    print("El modelo RandomForest de bikes no cumple la metrica identidad")


El modelo RandomForest de evaluación cumple la metrica identidad
El modelo Red neuronal de evaluación cumple la metrica identidad
El modelo Red neuronal de bikes cumple la metrica identidad
El modelo RandomForest de bikes cumple la metrica identidad


In [39]:
#Comprobaremos la metrica separabilidad
#Para comprobarla recorreremos 256 ejemplos en los cuatro modelos y los comparamos con otro ejemplo
ejemplo1=atributos[5552]
atributivos = atributos[5553:5809,:]
cumpleSeparabilidad = True
for x in atributivos:
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    exp2 = explain_model(forest_evaluation,ejemplo1,2,atributos)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo1,exp1,exp2)
    if(not cumpleSeparabilidad):
        break
if(cumpleSeparabilidad):
    print("El modelo RandomForest de evaluación cumple la metrica separabilidad")
else:
    print("El modelo RandomForest de evaluación no cumple la metrica separabilidad")

cumpleSeparabilidad = True
for x in atributivos:
    exp1 = explain_model(red_evaluation,x,2,atributos)
    exp2 = explain_model(red_evaluation,ejemplo1,2,atributos)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo1,exp1,exp2)
    if(not cumpleSeparabilidad):
        break

if(cumpleSeparabilidad):
    print("El modelo Red neuronal de evaluación cumple la metrica separabilidad")
else:
    print("El modelo Red neuronal de evaluación no cumple la metrica separabilidad")

ejemplo2 = attr[17121]
X=attr[17122:17378,:]
cumpleSeparabilidad=True
for x in X:
    exp1 = explain_model(red_bikes,x,2,attr)
    exp2 = explain_model(red_bikes,ejemplo2,2,attr)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo2,exp1,exp2)
    if(not cumpleSeparabilidad):
        break
    
if(cumpleSeparabilidad):
    print("El modelo Red neuronal de bikes cumple la metrica separabilidad")
else:
    print("El modelo Red neuronal de bikes no cumple la metrica separabilidad")
    
cumpleSeparabilidad=True
for x in X:
    exp1 = explain_model(forest_bikes,x,2,attr)
    exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo2,exp1,exp2)
    if(not cumpleSeparabilidad):
        break
    
if(cumpleSeparabilidad):
    print("El modelo RandomForest de bikes cumple la metrica separabilidad")
else:
    print("El modelo RandomForest de bikes no cumple la metrica separabilidad")


El modelo RandomForest de evaluación no cumple la metrica separabilidad
El modelo Red neuronal de evaluación no cumple la metrica separabilidad
El modelo Red neuronal de bikes no cumple la metrica separabilidad
El modelo RandomForest de bikes no cumple la metrica separabilidad
